# Asset Registry Blockchain Events Analysis

This notebook analyzes blockchain events from the Asset Registry smart contract for the last 1,000 blocks.

## Objectives:
1. Query all blockchain events related to the deployed contract for the last 1,000 blocks
2. Generate analytics:
   - Total number of assets ever registered
   - Total number of ownership transfers
   - Top 3 most active owners (by number of transfers)
3. Export analysis to JSON and Markdown
4. Create visualizations showing activity trends over time



In [ ]:
# Import required libraries
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import json
from collections import Counter

# Configuration
API_BASE_URL = "http://localhost:3000/api"

print("Libraries imported successfully!")
print(f"API Base URL: {API_BASE_URL}")



## Step 1: Query Events from Last 1,000 Blocks



In [ ]:
# Query events from the last 1,000 blocks
def get_recent_events(blocks=1000):
    """Fetch events from the last N blocks"""
    url = f"{API_BASE_URL}/events/recent"
    params = {"blocks": blocks}
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        if data.get('success'):
            return data
        else:
            print(f"Error: {data.get('error', 'Unknown error')}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching events: {e}")
        return None

# Fetch the data
print("Fetching events from last 1,000 blocks...")
events_data = get_recent_events(1000)

if events_data:
    print(f"✓ Successfully fetched events")
    print(f"  Block range: {events_data['blockRange']['from']} to {events_data['blockRange']['to']}")
    print(f"  Assets found: {events_data['assets']['count']}")
    print(f"  Transfers found: {events_data['transfers']['count']}")
else:
    print("✗ Failed to fetch events. Make sure the backend server is running.")



## Step 2: Get All Assets (for Complete Analytics)

In [ ]:
# Get all assets ever registered (not just from last 1,000 blocks)
def get_all_assets():
    """Fetch all registered assets"""
    url = f"{API_BASE_URL}/assets"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        if data.get('success'):
            return data['data']
        else:
            print(f"Error: {data.get('error', 'Unknown error')}")
            return []
    except requests.exceptions.RequestException as e:
        print(f"Error fetching assets: {e}")
        return []

# Fetch all assets
print("Fetching all registered assets...")
all_assets = get_all_assets()
print(f"✓ Total assets ever registered: {len(all_assets)}")



## Step 3: Get All Transfers (for Complete Analytics)



In [ ]:
# Get all transfers using the efficient endpoint
def get_all_transfers():
    """Fetch all transfers from the API"""
    url = f"{API_BASE_URL}/transfers"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        if data.get('success'):
            return data['data']
        else:
            print(f"Error: {data.get('error', 'Unknown error')}")
            return []
    except requests.exceptions.RequestException as e:
        print(f"Error fetching transfers: {e}")
        return []

# Fetch all transfers
print("Fetching all transfers...")
all_transfers = get_all_transfers()
print(f"✓ Total transfers found: {len(all_transfers)}")



## Step 4: Data Preparation and Analysis



In [ ]:
# Convert to DataFrames for easier analysis
assets_df = pd.DataFrame(all_assets)
transfers_df = pd.DataFrame(all_transfers)

# Convert date columns
if not assets_df.empty and 'registeredAt' in assets_df.columns:
    assets_df['registeredAt'] = pd.to_datetime(assets_df['registeredAt'])

if not transfers_df.empty and 'transferredAt' in transfers_df.columns:
    transfers_df['transferredAt'] = pd.to_datetime(transfers_df['transferredAt'])

print("DataFrames created:")
print(f"Assets DataFrame: {len(assets_df)} rows")
print(f"Transfers DataFrame: {len(transfers_df)} rows")

if not assets_df.empty:
    print("\nAssets DataFrame columns:", assets_df.columns.tolist())
if not transfers_df.empty:
    print("Transfers DataFrame columns:", transfers_df.columns.tolist())



In [ ]:
# Calculate analytics

# 1. Total number of assets ever registered
total_assets = len(all_assets)

# 2. Total number of ownership transfers
# Exclude initial registrations (where fromOwner is null)
ownership_transfers = [t for t in all_transfers if t.get('fromOwner') is not None]
total_transfers = len(ownership_transfers)

# 3. Top 3 most active owners (by number of transfers)
# Count transfers where owner is either fromOwner or toOwner
owner_activity = Counter()

for transfer in all_transfers:
    from_owner = transfer.get('fromOwner')
    to_owner = transfer.get('toOwner')
    
    if from_owner:
        owner_activity[from_owner] += 1
    if to_owner:
        owner_activity[to_owner] += 1

# Get top 3 most active owners
top_owners = owner_activity.most_common(3)

# Prepare analytics dictionary
analytics = {
    "analysis_date": datetime.now().isoformat(),
    "block_range": {
        "from": events_data['blockRange']['from'] if events_data else None,
        "to": events_data['blockRange']['to'] if events_data else None,
        "blocks_analyzed": events_data['blockRange']['blocks'] if events_data else None
    },
    "total_assets_registered": total_assets,
    "total_ownership_transfers": total_transfers,
    "top_3_active_owners": [
        {
            "address": owner,
            "transfer_count": count
        }
        for owner, count in top_owners
    ],
    "summary": {
        "assets_in_last_1000_blocks": events_data['assets']['count'] if events_data else 0,
        "transfers_in_last_1000_blocks": events_data['transfers']['count'] if events_data else 0
    }
}

print("Analytics calculated:")
print(f"  Total assets registered: {total_assets}")
print(f"  Total ownership transfers: {total_transfers}")
print(f"  Top 3 active owners:")
for i, (owner, count) in enumerate(top_owners, 1):
    print(f"    {i}. {owner}: {count} transfers")



## Step 5: Visualizations - Activity Trends Over Time



In [ ]:
# Create visualizations showing activity trends over time

# Prepare data for time series analysis
if not transfers_df.empty and 'transferredAt' in transfers_df.columns:
    # Group transfers by date
    transfers_df['date'] = pd.to_datetime(transfers_df['transferredAt']).dt.date
    daily_transfers = transfers_df.groupby('date').size().reset_index(name='count')
    daily_transfers['date'] = pd.to_datetime(daily_transfers['date'])
    
    # Group assets by registration date
    if not assets_df.empty and 'registeredAt' in assets_df.columns:
        assets_df['date'] = pd.to_datetime(assets_df['registeredAt']).dt.date
        daily_assets = assets_df.groupby('date').size().reset_index(name='count')
        daily_assets['date'] = pd.to_datetime(daily_assets['date'])
    else:
        daily_assets = pd.DataFrame(columns=['date', 'count'])
else:
    daily_transfers = pd.DataFrame(columns=['date', 'count'])
    daily_assets = pd.DataFrame(columns=['date', 'count'])

print("Time series data prepared for visualization")



In [ ]:
# Create activity trends chart
fig, axis = plt.subplots(2, 1, figsize=(14, 10))

# Chart 1: Daily Asset Registrations
if not daily_assets.empty:
    axis[0].plot(daily_assets['date'], daily_assets['count'], 
                 marker='o', linewidth=2, markersize=6, color='#2ecc71', label='Asset Registrations')
    axis[0].fill_between(daily_assets['date'], daily_assets['count'], alpha=0.3, color='#2ecc71')
    axis[0].set_title('Daily Asset Registrations Over Time', fontsize=14, fontweight='bold', pad=20)
    axis[0].set_xlabel('Date', fontsize=12)
    axis[0].set_ylabel('Number of Assets Registered', fontsize=12)
    axis[0].grid(True, alpha=0.3, linestyle='--')
    axis[0].legend(fontsize=10)
    axis[0].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    axis[0].xaxis.set_major_locator(mdates.DayLocator(interval=max(1, len(daily_assets) // 10)))
    plt.setp(axis[0].xaxis.get_majorticklabels(), rotation=45, ha='right')
else:
    axis[0].text(0.5, 0.5, 'No asset registration data available', 
                ha='center', va='center', transform=axis[0].transaxis, fontsize=12)
    axis[0].set_title('Daily Asset Registrations Over Time', fontsize=14, fontweight='bold')

# Chart 2: Daily Ownership Transfers
if not daily_transfers.empty:
    axis[1].plot(daily_transfers['date'], daily_transfers['count'], 
                 marker='s', linewidth=2, markersize=6, color='#3498db', label='Ownership Transfers')
    axis[1].fill_between(daily_transfers['date'], daily_transfers['count'], alpha=0.3, color='#3498db')
    axis[1].set_title('Daily Ownership Transfers Over Time', fontsize=14, fontweight='bold', pad=20)
    axis[1].set_xlabel('Date', fontsize=12)
    axis[1].set_ylabel('Number of Transfers', fontsize=12)
    axis[1].grid(True, alpha=0.3, linestyle='--')
    axis[1].legend(fontsize=10)
    axis[1].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    axis[1].xaxis.set_major_locator(mdates.DayLocator(interval=max(1, len(daily_transfers) // 10)))
    plt.setp(axis[1].xaxis.get_majorticklabels(), rotation=45, ha='right')
else:
    axis[1].text(0.5, 0.5, 'No transfer data available', 
                ha='center', va='center', transform=axis[1].transaxis, fontsize=12)
    axis[1].set_title('Daily Ownership Transfers Over Time', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('activity_trends.png', dpi=300, bbox_inches='tight')
print("✓ Chart saved as 'activity_trends.png'")
plt.show()



In [ ]:
# Create a combined activity chart
if not daily_assets.empty or not daily_transfers.empty:
    fig, ax = plt.subplots(figsize=(14, 7))
    
    if not daily_assets.empty:
        ax.plot(daily_assets['date'], daily_assets['count'], 
               marker='o', linewidth=2.5, markersize=8, color='#2ecc71', 
               label='Asset Registrations', alpha=0.8)
    
    if not daily_transfers.empty:
        ax.plot(daily_transfers['date'], daily_transfers['count'], 
               marker='s', linewidth=2.5, markersize=8, color='#3498db', 
               label='Ownership Transfers', alpha=0.8)
    
    ax.set_title('Combined Activity Trends: Asset Registrations and Ownership Transfers', 
                fontsize=16, fontweight='bold', pad=20)
    ax.set_xlabel('Date', fontsize=13)
    ax.set_ylabel('Number of Events', fontsize=13)
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.legend(fontsize=11, loc='best')
    
    if not daily_assets.empty:
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        ax.xaxis.set_major_locator(mdates.DayLocator(interval=max(1, len(daily_assets) // 10)))
    elif not daily_transfers.empty:
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        ax.xaxis.set_major_locator(mdates.DayLocator(interval=max(1, len(daily_transfers) // 10)))
    
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig('combined_activity_trends.png', dpi=300, bbox_inches='tight')
    print("✓ Combined chart saved as 'combined_activity_trends.png'")
    plt.show()
else:
    print("No data available for combined chart")



## Step 6: Export Analytics to JSON



In [ ]:
# Export analytics to JSON file
with open('analytics.json', 'w') as f:
    json.dump(analytics, f, indent=2, default=str)

print("✓ Analytics exported to 'analytics.json'")
print("\nAnalytics Summary:")
print(json.dumps(analytics, indent=2, default=str))



## Step 7: Generate Markdown Summary



In [ ]:
# Generate Markdown summary
def generate_markdown_summary(analytics_data):
    """Generate a markdown summary of the analytics"""
    
    md_content = f"""# Asset Registry Blockchain Events Analysis Summary

**Analysis Date:** {analytics_data['analysis_date']}

## Overview

This analysis covers blockchain events from the Asset Registry smart contract for the last 1,000 blocks.

**Block Range Analyzed:**
- From Block: {analytics_data['block_range']['from']}
- To Block: {analytics_data['block_range']['to']}
- Total Blocks: {analytics_data['block_range']['blocks']}

## Key Metrics

### Total Assets Registered
**{analytics_data['total_assets_registered']}** assets have been registered since the contract deployment.

### Total Ownership Transfers
**{analytics_data['total_ownership_transfers']}** ownership transfers have occurred (excluding initial registrations).

### Activity in Last 1,000 Blocks
- Assets registered: {analytics_data['summary']['assets_in_last_1000_blocks']}
- Transfers occurred: {analytics_data['summary']['transfers_in_last_1000_blocks']}

## Top 3 Most Active Owners

The following addresses have been the most active in terms of transfer activity:

"""
    
    for i, owner_data in enumerate(analytics_data['top_3_active_owners'], 1):
        md_content += f"{i}. **{owner_data['address']}**\n"
        md_content += f"   - Transfer Count: {owner_data['transfer_count']}\n\n"
    
    if len(analytics_data['top_3_active_owners']) == 0:
        md_content += "*No transfer activity data available.*\n\n"
    
    md_content += """## Visualizations

Activity trend charts have been generated showing:
- Daily asset registrations over time
- Daily ownership transfers over time
- Combined activity trends

Charts are saved as PNG files:
- `activity_trends.png`: Separate charts for registrations and transfers
- `combined_activity_trends.png`: Combined view of all activity

## Data Sources

All data was queried from the Asset Registry Backend API:
- Endpoint: `GET /api/events/recent?blocks=1000`
- Endpoint: `GET /api/assets`
- Endpoint: `GET /api/transfers`

## Notes

- Initial asset registrations are not counted as transfers
- Transfer counts include both sending and receiving transfers for each owner
- All addresses are normalized to lowercase for consistency
"""
    
    return md_content

# Generate and save markdown summary
markdown_summary = generate_markdown_summary(analytics)

with open('summary.md', 'w') as f:
    f.write(markdown_summary)

print("✓ Markdown summary exported to 'summary.md'")
print("\n" + "="*60)
print(markdown_summary)
print("="*60)



## Analysis Complete! ✓

All analytics have been generated and exported:
- ✅ `analytics.json` - Complete analytics data in JSON format
- ✅ `summary.md` - Human-readable summary in Markdown format
- ✅ Activity trend charts saved as PNG files

